In [125]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [126]:
## Load the dataset

data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [127]:
## Preprocess the data
### Drop the irrelevant columns

data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [128]:
lbl_encoder_gender = LabelEncoder()
data['Gender'] = lbl_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [129]:
## Onehot Encode Geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [130]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [131]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [132]:
## Combine the OHE columnd with original data

data = pd.concat([data.drop('Geography', axis =1), geo_encoded_df], axis =1 )

In [133]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [134]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(lbl_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)


In [135]:
## Divide the dataset into independent and dependent features

X = data.drop('Exited', axis=1)
y = data['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [136]:
## Scaling the features
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [137]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

### ANN Implementations

In [138]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [139]:
## Build our ANN Model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), # 1st Hidden Layer ( Connected with Input Layer )
    Dense(32, activation='relu'), # HL 2
    Dense(1,activation= 'sigmoid') # Output Layer
])

In [140]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                832       
                                                                 
 dense_13 (Dense)            (None, 32)                2080      
                                                                 
 dense_14 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [141]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [142]:
## compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy']) 

In [143]:
## Set up the tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

In [144]:
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [145]:
## Set up Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [146]:
### Train the model
history = model.fit(
    X_train, y_train, validation_data =(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback] 
)

Epoch 1/100
250/250 [==============================] - 0s 673us/step - loss: 0.4031 - accuracy: 0.8339 - val_loss: 0.3539 - val_accuracy: 0.8545
Epoch 2/100
250/250 [==============================] - 0s 464us/step - loss: 0.3551 - accuracy: 0.8565 - val_loss: 0.3520 - val_accuracy: 0.8585
Epoch 3/100
250/250 [==============================] - 0s 463us/step - loss: 0.3511 - accuracy: 0.8560 - val_loss: 0.3434 - val_accuracy: 0.8555
Epoch 4/100
250/250 [==============================] - 0s 456us/step - loss: 0.3432 - accuracy: 0.8574 - val_loss: 0.3467 - val_accuracy: 0.8550
Epoch 5/100
250/250 [==============================] - 0s 454us/step - loss: 0.3433 - accuracy: 0.8584 - val_loss: 0.3482 - val_accuracy: 0.8610
Epoch 6/100
250/250 [==============================] - 0s 464us/step - loss: 0.3390 - accuracy: 0.8616 - val_loss: 0.3480 - val_accuracy: 0.8540
Epoch 7/100
250/250 [==============================] - 0s 470us/step - loss: 0.3342 - accuracy: 0.8636 - val_loss: 0.3498 - val_ac

In [147]:
model.save('model.h5')

/Users/deepayanbose/Documents/AIML_COHORT/venv/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [148]:
## Load TensorBoard extension
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [149]:
import numpy as np

In [150]:
from tensorflow.keras.models import load_model

In [151]:
## load the trained model , scaler pickle , and onehotencoder 
model = load_model('model.h5')

In [152]:
## load the encoders and scaler

with open('label_encoder_gender.pkl', 'rb') as file:
    lbl_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)



In [153]:
# Example input data

input_data = {
    'CreditScore' : 600,
    'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}

In [154]:
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

/Users/deepayanbose/Documents/AIML_COHORT/venv/lib/python3.10/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [155]:
input_data = pd.DataFrame([input_data])
input_data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [156]:
input_data = pd.concat([input_data.reset_index(drop=True), geo_encoded_df], axis=1)

In [157]:
input_data['Gender'] = lbl_encoder_gender.transform(input_data['Gender'])

In [158]:
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [159]:
input_data.drop(columns=['Geography'], axis=1, inplace=True)

In [160]:
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [161]:
input_scaled = scaler.transform(input_data)

In [162]:
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [163]:
prediction = model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 66ms/step


array([[0.03613272]], dtype=float32)

In [164]:
prediction_proba = prediction[0][0]

In [165]:
prediction_proba

0.03613272

In [166]:
print("yes" if prediction_proba > 0.5 else "no")

no
